In [1]:
import asyncio
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [2]:
data = {
    "restaurant": [
        "استقبل المطعم أكثر من 200 طلب خلال اليوم.",
        "تم تحديث قائمة الطعام بإضافة وجبات نباتية جديدة.",
        "نفد مخزون الأرز وتم إرسال تنبيه إلى إدارة المشتريات.",
        "أُضيف موظف جديد إلى قسم التوصيل.",
        "تم تسجيل زيادة في الطلبات خلال عطلة نهاية الأسبوع بنسبة 35%.",
        "اكتشف النظام تأخيراً في توصيل طلبات الحي الجنوبي.",
        "تمت مزامنة بيانات المبيعات مع نظام المحاسبة تلقائياً.",
        "أُجري تحليل لأكثر الأطباق مبيعاً خلال الشهر.",
        "تم إصدار تقرير بالأرباح الأسبوعية وإرساله إلى الإدارة.",
        "بدأ النظام بعملية الجرد الآلي للمكونات الغذائية.",
        "تمت الموافقة على طلب شراء مكونات جديدة للمطعم.",
        "أبلغ النظام عن نقص في بعض المكونات الأساسية مثل الجبن والزيت.",
        "جرى ربط النظام بخدمة توصيل جديدة لتحسين سرعة الطلبات.",
        "أُضيفت خاصية تتبع الطباخين لتقييم الأداء في الوقت الفعلي.",
        "أنشأ المدير حملة خصومات عبر النظام لزيادة المبيعات في المساء.",
    ],
    "sales": [
        "تم تسجيل 50 صفقة بيع جديدة هذا الأسبوع.",
        "أنشأ النظام تقريراً مفصلاً عن أداء مندوبي المبيعات.",
        "تمت مزامنة بيانات العملاء المحتملين مع نظام إدارة العملاء.",
        "انخفضت المبيعات في الربع الأخير بنسبة 12% مقارنة بالربع السابق.",
        "تم إطلاق حملة تسويقية رقمية لزيادة الوعي بالمنتجات الجديدة.",
        "حدد النظام 10 فرص مبيعات جديدة بناءً على تحليل السوق.",
        "تم تحديث قائمة الأسعار لتتناسب مع التغيرات في السوق.",
        "أُرسلت إشعارات لمندوبي المبيعات حول العملاء غير المتابعين.",
        "حقق الفريق المستهدف نسبة إنجاز بلغت 95% من الأهداف الشهرية.",
        "تم تحليل بيانات المنافسين وإضافة التوصيات الاستراتيجية.",
        "أنشأ النظام مخططاً بيانياً للمبيعات اليومية حسب المنطقة.",
        "تم اكتشاف تراجع في مبيعات منتج محدد واقتُرح خصم ترويجي.",
        "جرى دمج النظام مع تطبيق نقاط الولاء لتعزيز المبيعات المتكررة.",
        "تم إعداد تقرير الأرباح والخسائر وإرساله إلى الإدارة العليا.",
        "تمت أرشفة بيانات المبيعات القديمة لتحسين أداء النظام.",
    ],
    "customers": [
        "تمت إضافة 120 عميلاً جديداً خلال الشهر الحالي.",
        "أرسل النظام استبيان رضا العملاء بعد كل عملية شراء.",
        "تم تحديث بيانات التواصل لعدد من العملاء القدامى.",
        "أظهر التحليل أن نسبة الولاء ارتفعت بنسبة 15%.",
        "قام فريق الدعم بحل 95% من الشكاوى خلال أول 24 ساعة.",
        "أطلق النظام حملة شكر للعملاء المميزين.",
        "تم تصنيف العملاء حسب سلوك الشراء وتكرار التعامل.",
        "تلقى العميل تنبيهًا بشأن عرض خاص على المنتجات التي يفضلها.",
        "تم إنشاء تقارير شهرية لتقييم رضا العملاء.",
        "قام النظام بدمج بيانات العملاء من وسائل التواصل الاجتماعي.",
        "تم تحديد العملاء ذوي الإنفاق العالي وإرسال عروض مخصصة لهم.",
        "جرى تحليل تفاعلات العملاء لتحديد الاتجاهات الجديدة.",
        "أضاف النظام خاصية التوصية الذكية بالعروض المناسبة لكل عميل.",
        "تم تنفيذ حملة استرجاع العملاء غير النشطين.",
        "أصدر النظام تنبيهًا بانخفاض معدل التفاعل في منطقة محددة.",
    ],
    "real_estate": [
        "تمت إضافة 30 وحدة عقارية جديدة إلى النظام.",
        "انتهى عقد الإيجار لخمسة مستأجرين وتم إرسال إشعارات التجديد.",
        "تم تسجيل دفعات الإيجار لشهر أكتوبر تلقائياً.",
        "بدأت عملية الصيانة الدورية للعقارات السكنية.",
        "أضاف النظام ميزة تتبع حالة العقود النشطة والمنتهية.",
        "تم إصدار تقرير بالأرباح الشهرية للعقارات المؤجرة.",
        "جرى تحديث بيانات الملاك والمستأجرين في قاعدة البيانات.",
        "تم اكتشاف تأخير في سداد دفعات أحد المستأجرين.",
        "أرسل النظام إشعاراً للفريق المالي حول العقود المتأخرة.",
        "تم جدولة زيارة تفقدية لأحد الأبراج التجارية.",
        "أضاف النظام خريطة تفاعلية لعرض مواقع العقارات.",
        "تم ربط النظام مع بوابة الدفع الإلكتروني لتسريع التحصيل.",
        "جرى تحليل نسبة الإشغال للعقارات خلال الربع الأخير.",
        "أصدر النظام توصية برفع الإيجار لعقارات ذات طلب مرتفع.",
        "تم أرشفة العقود المنتهية وتحويلها إلى السجل التاريخي.",
    ],
}


texts, labels = [], []
label2id = {label: idx for idx, label in enumerate(data.keys())}
for label, examples in data.items():
    texts.extend(examples)
    labels.extend([label2id[label]] * len(examples))

print(f"Total samples: {len(texts)} | Classes: {label2id}")

Total samples: 60 | Classes: {'restaurant': 0, 'sales': 1, 'customers': 2, 'real_estate': 3}


In [3]:
from cohere import AsyncClientV2


def normalize_embeddings(embeddings):
    norm = torch.norm(torch.tensor(embeddings), dim=1, keepdim=True)
    return (torch.tensor(embeddings) / norm).tolist()


class BaseEmbeddings:
    pass


class CohereEmbeddings(BaseEmbeddings):
    def __init__(self, cohere_client):
        self.cohere_client = cohere_client

    async def embed(
        self,
        list_of_text: list[str],
        model_name="embed-v4.0",
        batch_size=10,
    ) -> list[list[float]]:
        vectors = []
        for i in range(0, len(list_of_text), batch_size):
            batch = list_of_text[i : i + batch_size]
            response = await self.cohere_client.embed(
                texts=batch,
                model=model_name,
                input_type="search_document",
                embedding_types=["float"],
                output_dimension=1024,
            )
            batch_vectors = response.embeddings.float
            normalized = normalize_embeddings(batch_vectors)
            vectors.extend(normalized)
        return vectors


async def generate_embeddings():
    cohere_client = AsyncClientV2("q5K7ogyRXviI1pCR4PeTqvvjFyVGLkHvfwlv6EmE")
    embedder = CohereEmbeddings(cohere_client)
    embeddings = await embedder.embed(texts)
    return torch.tensor(embeddings, dtype=torch.float32), torch.tensor(labels)

In [4]:
class EmbeddingClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes, hidden_dim=512, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, num_classes),
        )

    def forward(self, x):
        return self.net(x)

In [5]:
async def main():
    X, y = await generate_embeddings()
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    model = EmbeddingClassifier(embedding_dim=1024, num_classes=4)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    num_epochs = 10
    epoch_pbar = tqdm(range(num_epochs), desc="Training Progress")

    for epoch in epoch_pbar:
        model.train()
        running_loss = 0.0
        batch_pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for X_batch, y_batch in batch_pbar:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})
        avg_epoch_loss = running_loss / len(dataloader)
        epoch_pbar.set_postfix({"avg_epoch_loss": f"{avg_epoch_loss:.4f}"})
    return model

In [6]:
model = await main()

Training Progress: 100%|██████████| 10/10 [00:01<00:00,  8.79it/s, avg_epoch_loss=0.0161]


In [7]:
label_map = {0: "restaurant", 1: "sales", 2: "customers", 3: "real_estate"}


async def classify_query(
    query: str, model, embedder: CohereEmbeddings, label_map: dict
):
    query_emb = await embedder.embed([query])
    query_tensor = torch.tensor(query_emb, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        logits = model(query_tensor)
        probs = F.softmax(logits, dim=1)
        predicted_idx = probs.argmax(dim=1).item()

    predicted_label = label_map[predicted_idx]
    confidence = probs[0][predicted_idx].item()

    return predicted_label, confidence


cohere_client = AsyncClientV2("q5K7ogyRXviI1pCR4PeTqvvjFyVGLkHvfwlv6EmE")
embedder = CohereEmbeddings(cohere_client)
label_map = {0: "restaurant", 1: "sales", 2: "customers", 3: "real_estate"}
query = "أطلقت الشركة حملة ترويجية جديدة لزيادة المبيعات."
label, conf = await classify_query(query, model, embedder, label_map)
print(f"📊 Class: {label} | 🔥 Confidence: {conf:.2f}")

📊 Class: sales | 🔥 Confidence: 0.87
